In [ ]:
# load in the sqlite file into memory

database_file = "data/ap_sample.sqlite"

In [ ]:
# turn sqlite file into dictionary of the following form

'''
    {
        geometry_tuple (): {
            level: int,
            scan_data: [],
        }
    }
'''